In [ ]:
import numpy as np

repo_root = "/Users/miskodzamba/Dropbox/research/gits/spf/"
import sys

sys.path.append(repo_root)  # go to parent dir
# from boundaries import
from spf.gps.boundaries import (
    franklin_boundary,
    crissy_boundary,
    crissy_boundary_convex,
    franklin_safe,
)

from matplotlib import pyplot as plt

boundary = crissy_boundary_convex
plt.scatter(boundary[:, 0], boundary[:, 1])
for idx in range(boundary.shape[0]):
    plt.annotate(f"p{idx}", (boundary[idx, 0], boundary[idx, 1]))
plt.axis("equal")

In [ ]:
franklin_safe.mean(axis=0)

In [ ]:
import numpy


def degnorm(x):
    return ((x + 180.0) % 360.0) - 180.0


def pinorm(x):
    return ((x + np.pi) % (2 * np.pi)) - np.pi


# https://mapscaping.com/how-to-calculate-bearing-between-two-coordinates/
# p0 = [ long , lat ] # in radians
# p1 = [ long , lat ] # in radians
# return degrees from north that angle from p0->p1 makes
# dead north is 0deg, to the right is + and to the left is -
def calc_bearing(p0, p1):
    # Calculate the bearing
    p0_rad = np.deg2rad(p0)
    p1_rad = np.deg2rad(p1)
    bearing_rad = np.arctan2(
        np.sin(p1_rad[0] - p0_rad[0]) * np.cos(p1_rad[1]),
        np.cos(p0_rad[1]) * np.sin(p1_rad[1])
        - np.sin(p0_rad[1]) * np.cos(p1_rad[1]) * np.cos(p1_rad[0] - p0_rad[0]),
    )

    return np.rad2deg(pinorm(bearing_rad))  # np.rad2deg(bearing_rad)


"""
Craft is facing p0 from p1 and would like to go to p2
"""


def calc_relative_bearing(p_facing, p_current, p_desired):
    return (
        degnorm(calc_bearing(p_current, p_desired) - calc_bearing(p_current, p_facing)),
        calc_bearing(p_current, p_facing),
        calc_bearing(p_current, p_desired),
    )

In [ ]:
for start_idx in range(len(boundary) - 2):
    p0 = boundary[start_idx]
    p1 = boundary[start_idx + 1]
    p2 = boundary[start_idx + 2]
    b01 = calc_bearing(p0, p1)
    b12 = calc_bearing(p1, p2)
    print(start_idx, calc_relative_bearing(p_current=p0, p_facing=p1, p_desired=p2))

In [ ]:
import numpy as np
import yaml
import zarr
from pyproj import Proj

from spf.dataset.v5_data import v5rx_2xf64_keys, v5rx_f64_keys
from spf.gps.boundaries import franklin_safe  # crissy_boundary_convex
from spf.utils import zarr_new_dataset, zarr_open_from_lmdb_store, zarr_shrink


def lat_lon_to_xy(lat, lon, center_lat, center_lon):
    proj_centered = Proj(proj="aeqd", lat_0=center_lat, lon_0=center_lon, datum="WGS84")
    return proj_centered(lon, lat)


# lat_lon_to_xy(
#                     lat=gps_lat,
#                     lon=gps_long,
#                     center_lat=gps_center_long_lat[1],
#                     center_lon=gps_center_long_lat[0],
#                 )

In [ ]:
franklin_safe.mean(axis=0)

In [ ]:
gps_center_long, gps_center_lat = franklin_safe.mean(axis=0)
lat_lon_to_xy(
    lat=gps_center_lat - 0.0001,
    lon=gps_center_long,
    center_lat=gps_center_lat,
    center_lon=gps_center_long,
)

In [ ]:
near_bridge = (-122.39219619079822, 37.7851203550129)

In [ ]:
np.array(
    lat_lon_to_xy(
        lat=near_bridge[1],
        lon=near_bridge[0],
        center_lat=gps_center_lat,
        center_lon=gps_center_long,
    )
) * 10000

In [ ]:
gps_center_long

In [ ]:
from spf.motion_planners.dynamics import Dynamics
from spf.motion_planners.planner import BouncePlanner
from spf.gps.boundaries import fort_baker_right_boundary, fort_baker_left_boundary
import numpy as np

boundary = fort_baker_left_boundary
bp = BouncePlanner(
    dynamics=Dynamics(
        bounding_box=boundary,
        bounds_radius=0.000000001,
    ),
    start_point=boundary.mean(axis=0),
    epsilon=0.0000001,
    step_size=0.00001,
)

n = 20000
p = bp.yield_points()
z = np.array([next(p) for x in range(n)])

# assert (np.unique(z).size / n) > 0.9

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(boundary[:, 0], boundary[:, 1])
plt.scatter(z[:, 0], z[:, 1], s=0.1)